# Byte-pair enconding

In [ ]:
# initialized vocabulary will all unique words in the corpus and their counts. Calculate also pair counts, and update vocabulary iterativelly

from collections import defaultdict

'''
generates a vocabulary that represents words as a sequence of characters and tracks their counts
'''
def initialize_vocabulary(corpus):
    vocabulary=defaultdict(int)
    charset = set()
    for word in corpus:
        word_with_marker = "_" + word
        characters = list(word_with_marker)
        charset.update(characters)
        tokenized_word = ' '.join(characters)
        vocabulary[tokenized_word] += 1
    return vocabulary, charset

In [ ]:
# merge more frequency pair of tokens

'''
How often adjacent token pairs appear in the tokenized vocabulary words '''
def get_pair_counts(vocabulary):
    pair_counts = defaultdict(int)
    for tokenized_word, count in vocabulary.items():
        tokens = tokenized_word.split()
        for i in range(len(tokens)-1):
            pair = (tokens[i], tokens[i+1])
            pair_counts[pair] += count
    return pair_counts

In [ ]:
# merge pairs

def merge_pair(vocabulary, pair):
    new_vocabulary = {}
    bigram = re.escape(' '.join(pair))
    pattern = re.compile(r"(?<!\S)" + bigram + r"(?!\S)")
    for tokenized_word, count in vocabulary.items():
        new_tokenized_word = pattern.sub("".join(pair), tokenized_word)
        new_vocabulary[new_tokenized_word] = count
    return new_vocabulary
